# Project l’impact des aliment sur l’environnement, de la production à la mise en vente

In [ ]:
import pandas as pd
import projet_craber as craber
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df_ing = pd.read_csv("agribalyse-31-detail-par-ingredient.csv")
df_eta = pd.read_csv("agribalyse-31-detail-par-etape.csv")
df_syn = pd.read_csv("agribalyse-31-synthese.csv")

print(df_ing.shape)
print(df_eta.shape)
print(df_syn.shape)

## Pretraitement des données 

In [ ]:
df_synt = df_syn.drop_duplicates(subset=["Code CIQUAL"], keep="first")
df_etape = df_eta.drop_duplicates(subset=["Code CIQUAL"], keep="first")

In [ ]:
# remplissage des valeurs nulles
float_columns = df_synt.select_dtypes(include="float64").columns
float_columns_eta = df_eta.select_dtypes(include="float64").columns

# Traversée la data frame, remplissez les valeurs nulles comme moye
for column in float_columns:
    mean_value = df_synt[column].mean()
    df_synt[column].fillna(mean_value, inplace=True)

for column in float_columns_eta:
    mean_value = df_eta[column].mean()
    df_eta[column].fillna(mean_value, inplace=True)

# supprimer les lignes avec des strings vide
df_synt = df_synt.dropna()
df_eta = df_eta.dropna()


# Définir un seuil de 1.96 écarts types (intervalle de confiance à 95 %)


# threshold = 1.96

# # Calculer la moyenne et l'écart type
# mean = np.mean(df_synt["DQR"])
# std = np.std(df_synt["DQR"])

# # Calculer les bornes de l'intervalle de confiance
# low_bound = mean - std * threshold

# Filtrer les lignes avec des valeurs dans l'intervalle de confiance
df_synt = df_synt[df_synt["DQR"] <= 4]

# # Définir un seuil de 3 écarts types
# threshold = 3
# float_columns = float_columns.drop("Score unique EF", errors="ignore")
# float_columns = float_columns.drop("DQR", errors="ignore")

# for column in float_columns:
#     mean = np.mean(df_synt[column])
#     std = np.std(df_synt[column])
#     low_bound = mean - std * threshold
#     upper_bound = mean + std * threshold

#     df_synt[column] = np.where(df_synt[column] < low_bound, low_bound, df_synt[column])
#     df_synt[column] = np.where(
#         df_synt[column] > upper_bound, upper_bound, df_synt[column]
#     )

In [ ]:
df_synt.columns

In [ ]:
# Histogramme du 'Score unique EF'
plt.figure(figsize=(10, 6))
sns.histplot(df_synt["Score unique EF"], kde=True)
plt.title("Distribution du Score unique EF")
plt.xlabel("Score unique EF")
plt.ylabel("Fréquence")
plt.show()

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from warnings import filterwarnings


scaler = MinMaxScaler()
X = df_synt.iloc[:, 12:-3]
X = scaler.fit_transform(X)
X = pd.DataFrame(X, columns=df_synt.columns[12:-3])

In [ ]:
filterwarnings("ignore")
inertia = []
for k in range(1, 20):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(X.values.reshape(-1, 1))
    inertia.append(kmeans.inertia_)
plt.figure()
plt.plot(range(1, 20), inertia, marker="o")
plt.title("Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")
plt.show()

In [ ]:
tab20 = sns.color_palette("tab20", as_cmap=True)

tsne = TSNE()
tsne_data = tsne.fit_transform(X)

km = KMeans(n_clusters=5)
pred = km.fit_predict(X)

# afficher avec des couleur et des legendes
df = pd.DataFrame(tsne_data, columns=["x", "y"])
df["cluster"] = pred
sns.scatterplot(data=df, x="x", y="y", hue="cluster", palette=tab20)
plt.show()

In [ ]:
df_synt["cluster"] = pred  # Ajoutez les prédictions de cluster au DataFrame

# Créez un dictionnaire pour stocker les aliments par cluster
clusters = {}
for i in range(km.n_clusters):
    # Sélectionner les noms des aliments et les scores EF pour chaque cluster
    clusters[i] = (
        df_synt[df_synt["cluster"] == i][["Sous-groupe d'aliment", "Score unique EF"]]
        .drop_duplicates()
        .values.tolist()
    )

# Calculer la moyenne des scores EF pour chaque cluster
moyennes_clusters = {}
for cluster, aliments_scores in clusters.items():
    # Calculer la moyenne des scores EF pour le cluster actuel
    moyenne_ef = sum(score for _, score in aliments_scores) / len(aliments_scores)
    moyennes_clusters[cluster] = moyenne_ef

In [ ]:
clusters

In [ ]:
moyennes_clusters

In [ ]:
# Supposons que 'moyennes_clusters' est votre dictionnaire contenant les moyennes des scores EF par cluster
cluster_labels = list(clusters.keys())
aliments_par_cluster = [len(clusters[cluster]) for cluster in cluster_labels]
scores_moyens_ef = [moyennes_clusters[cluster] for cluster in cluster_labels]

fig, ax1 = plt.subplots(figsize=(10, 6))

# Créer le diagramme à barres pour le nombre d'aliments par cluster
ax1.bar(
    cluster_labels, aliments_par_cluster, color="skyblue", label="Nombre d'Aliments"
)
ax1.set_xlabel("Numéro du Cluster")
ax1.set_ylabel("Nombre d'Aliments", color="skyblue")
ax1.tick_params(axis="y", labelcolor="skyblue")
ax1.set_title("Nombre d'Aliments et Score EF Moyen par Cluster")

# Créer un second axe pour les scores EF moyens
ax2 = ax1.twinx()
ax2.plot(
    cluster_labels, scores_moyens_ef, color="red", label="Score EF Moyen", marker="o"
)
ax2.set_ylabel("Score EF Moyen", color="red")
ax2.tick_params(axis="y", labelcolor="red")

# Ajouter une légende
fig.legend(loc="upper left", bbox_to_anchor=(0.9, 0.9))

plt.xticks(cluster_labels)
plt.show()

In [ ]:
# Find the key with the maximum value
key_max_value = max(moyennes_clusters, key=moyennes_clusters.get)

print(key_max_value)  # Outputs: 'b'

In [ ]:
clusters_uniques = {}
for cluster in clusters.keys():
    # Sélectionner les aliments uniques pour chaque cluster
    clusters_uniques[cluster] = df_synt[df_synt["cluster"] == cluster][
        "Sous-groupe d'aliment"
    ].unique()

# Afficher les aliments uniques pour chaque cluster
clusters_uniques[key_max_value]

In [ ]:
from collections import defaultdict

# Supposons que 'clusters' est votre dictionnaire contenant les aliments et leurs scores EF par cluster
# et que le cinquième cluster est indexé par 4 (si vous commencez à compter à partir de 0)
aliments_scores_cluster_5 = clusters[
    key_max_value
]  # Remplacez '4' par l'index approprié si nécessaire

# Créer un dictionnaire pour cumuler les scores EF et compter les occurrences des aliments de même nom
cumul_scores_ef = defaultdict(
    lambda: [0, 0]
)  # Le premier élément est la somme des scores, le second est le compteur
for aliment, score_ef in aliments_scores_cluster_5:
    cumul_scores_ef[aliment][0] += score_ef
    cumul_scores_ef[aliment][1] += 1

# Calculer la moyenne des scores EF pour les aliments de même nom
moyennes_scores_ef = {
    aliment: total_score / count
    for aliment, (total_score, count) in cumul_scores_ef.items()
}

# Convertir le dictionnaire des moyennes en une liste triée pour la visualisation
aliments_moyennes = sorted(
    moyennes_scores_ef.items(), key=lambda item: item[1], reverse=True
)

# Extraire les noms des aliments et les moyennes des scores EF
noms_aliments_moyennes = [aliment for aliment, _ in aliments_moyennes]
moyennes_ef = [moyenne for _, moyenne in aliments_moyennes]

plt.figure(figsize=(12, 8))
bars = plt.barh(noms_aliments_moyennes, moyennes_ef, color="green")
plt.xlabel("Moyenne du Score EF")
plt.title(
    f"Moyenne de l'Impact Environnemental des Aliments du cluster {key_max_value}"
)
plt.gca().invert_yaxis()  # Inverser l'axe y pour afficher le score le plus élevé en haut

# Ajouter des annotations sur chaque barre
for bar in bars:
    plt.text(
        bar.get_width(),
        bar.get_y() + bar.get_height() / 2,
        f"{bar.get_width():.2f}",
        va="center",
        ha="left",
        color="black",
    )

plt.tight_layout()
plt.show()

In [ ]:
df_cluster_5 = df_synt[df_synt["cluster"] == key_max_value]
df_etape_5 = df_etape.iloc[:, 6:-3]

# Save "Code CIQUAL" column in a variable
code_ciqual = df_etape["Code CIQUAL"]

# Insert "Code CIQUAL" column at position 0
df_etape_5.insert(0, "Code CIQUAL", code_ciqual)

# Perform the join
df_jointure = pd.merge(df_cluster_5, df_etape_5, on="Code CIQUAL")


print(df_cluster_5.shape)


print(df_jointure.shape)

In [ ]:
df_jointure.head()

In [ ]:
df_jointure.columns[12:29]

In [ ]:
from statsmodels.stats.multicomp import MultiComparison

scaler = MinMaxScaler()
df_join_norm = df_jointure.copy()
df_join_norm = scaler.fit_transform(df_join_norm.iloc[:, 12:29])
df_join_norm = pd.DataFrame(df_join_norm, columns=df_jointure.columns[12:29])
df_join_norm["Sous-groupe d'aliment"] = df_jointure["Sous-groupe d'aliment"]

mc = MultiComparison(
    df_join_norm["Changement climatique"], df_join_norm["Sous-groupe d'aliment"]
)

result = mc.tukeyhsd(alpha=0.05)

print(result.summary())

```markdown
D'après le résumé de l'analyse Tukey HSD que vous avez fourni, voici les produits à impact élevé sur le changement climatique :

1. **Boissons sans alcool vs Autres produits à base de viande** : Les boissons sans alcool ont un impact significativement plus faible sur le changement climatique que les autres produits à base de viande, avec une différence moyenne de -22.9834 et une valeur p de 0.001. Cela suggère que les autres produits à base de viande pourraient être une cible prioritaire pour réduire l'impact environnemental.

2. **Boissons sans alcool vs Viandes crues** : Les boissons sans alcool ont également un impact significativement plus faible que les viandes crues, avec une différence moyenne de 12.5092 et une valeur p de 0.0063. Les viandes crues pourraient donc aussi être considérées comme ayant un impact élevé.

3. **Boissons sans alcool vs Viandes cuites** : De même, les boissons sans alcool ont un impact significativement plus faible que les viandes cuites, avec une différence moyenne de 18.6723 et une valeur p de 0.0. Les viandes cuites sont donc un autre groupe à impact élevé.

4. **Viandes crues vs Viandes cuites** : Il y a une différence significative entre les viandes crues et les viandes cuites, avec une différence moyenne de 6.1632 et une valeur p de 0.0129. Cela indique que les viandes cuites ont un impact plus élevé que les viandes crues.

Les autres comparaisons n'ont pas montré de différences significatives, ce qui signifie que l'on ne peut pas conclure à un impact élevé basé sur ces données.

Pour guider les décisions en matière de politique alimentaire ou de recommandations de consommation, on pourrait envisager de promouvoir une consommation réduite des produits à base de viande, en particulier des viandes cuites, et d'encourager des alternatives avec un impact moindre, comme les boissons sans alcool. De plus, des efforts pourraient être faits pour améliorer les pratiques de production dans le secteur de la viande afin de réduire son empreinte carbone.


In [ ]:
# Calculer la moyenne du score EF pour chaque catégorie
moyennes_par_categorie = df_join_norm.groupby("Sous-groupe d'aliment")[
    "Changement climatique"
].mean()

# Préparer les données pour le diagramme en camembert
labels = moyennes_par_categorie.index
sizes = moyennes_par_categorie.values
colors = plt.cm.Paired(range(len(labels)))  # Générer des couleurs pour chaque catégorie

# Créer le diagramme en camembert
plt.figure(figsize=(10, 8))
plt.pie(sizes, labels=labels, colors=colors, autopct="%1.1f%%", startangle=140)
plt.axis("equal")  # Assurer que le camembert est un cercle
plt.title("Pourcentage d'Impact sur le Changement Climatique par Catégorie d'Aliments")
plt.show()

In [ ]:
# Création du diagramme en barresplt.figure(figsize=(10, 6))
moyennes_par_categorie.plot(kind="bar", color="skyblue")

# Ajouter des titres et des étiquettes
plt.title("Moyenne de Changement Climatique par Sous-groupe d'Aliments")
plt.xlabel("Sous-groupe d'Aliments")
plt.ylabel("Moyenne de Changement Climatique")
plt.xticks(
    rotation=90, ha="right"
)  # Rotation des étiquettes pour une meilleure lisibilité

plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
plt.show()

In [ ]:
df_subset = df_join_norm.iloc[:, 1:-1]
# Matrice de corrélation pour le sous-ensemble
correlation_matrix_subset = df_subset.corr()

# Heatmap des corrélations pour le sous-ensemble
plt.figure(figsize=(10, 6))
sns.heatmap(correlation_matrix_subset, annot=True, cmap="coolwarm")
plt.title("Matrice de corrélation pour le sous-ensemble de colonnes")
plt.show()

In [ ]:
from sklearn.decomposition import PCA

# Supposons que df_join_norm est votre DataFrame et contient les indicateurs d'impact environnemental
# Sélectionner les colonnes d'impact environnemental pour l'ACP
colonnes_impact = df_join_norm.columns[1:-1]

# Préparer les données pour l'ACP
X = df_join_norm[colonnes_impact]

# Initialiser l'ACP
pca = PCA()  # Utiliser tous les composantes pour l'exemple
X_r = pca.fit_transform(X)

summary_table = pd.DataFrame(
    columns=[
        "Dimension",
        "Explained Variance",
        "Explained Variance Ratio",
        "Cumulative Explained Variance Ratio",
    ]
)

# Créer une liste pour stocker les lignes du tableau récapitulatif
rows = []

# Calculer les proportions cumulatives de variance expliquée
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Remplir la liste avec les informations sur chaque dimension
for i in range(len(pca.explained_variance_)):
    rows.append(
        {
            "Dimension": i + 1,
            "Explained Variance": pca.explained_variance_[i],
            "Explained Variance Ratio": pca.explained_variance_ratio_[i],
            "Cumulative Explained Variance Ratio": cumulative_variance_ratio[i],
        }
    )

# Créer le DataFrame récapitulatif en concaténant les lignes
summary_table = pd.concat([pd.DataFrame([row]) for row in rows], ignore_index=True)

summary_table

In [ ]:
# Calculer les proportions cumulatives de variance expliquée
cumulative_variance_ratio_percent = pca.explained_variance_ratio_ * 100

# Tracer le graphique des proportions de variance expliquée en pourcentage
plt.figure(figsize=(10, 6))
plt.plot(
    range(1, len(pca.explained_variance_ratio_) + 1),
    pca.explained_variance_ratio_,
    marker="o",
    linestyle="-",
)
plt.title("Proportions de Variances Expliquées (en pourcentage)")
plt.xlabel("Nombre de Composantes Principales")
plt.ylabel("Pourcentage de Variance Expliquée")
plt.xticks(range(1, len(cumulative_variance_ratio_percent) + 1))
plt.grid(True)
plt.show()

In [ ]:
n = X.shape[0]  # nb of individuals
p = X.shape[1]  # nb of variables
print(n, "  ", p)
eigval = (n - 1) / n * pca.explained_variance_  # eigen values
sqrt_eigval = np.sqrt(eigval)
corvar = np.zeros((p, p))  # empty matrix for coordinates
for k in range(p):
    corvar[:, k] = pca.components_[k, :] * sqrt_eigval[k]
# on modifie pour avoir un dataframe
coordvar = pd.DataFrame({"id": X.columns, "COR_1": corvar[:, 0], "COR_2": corvar[:, 1]})
coordvar

In [ ]:
moyennes_par_categorie

In [ ]:
pca = PCA(n_components=3)  # Utiliser tous les composantes pour l'exemple
X_r = pca.fit_transform(X)


# Supposons que 'X_r' est le résultat de l'ACP et 'df_join_norm' contient les sous-groupes d'aliments
# Ajouter les sous-groupes d'aliments comme une nouvelle colonne dans les données réduites
X_r_df = pd.DataFrame(X_r, columns=["PC1", "PC2", "PC3"])
X_r_df["Sous-groupe d'aliment"] = df_join_norm["Sous-groupe d'aliment"].values

# Calculer les moyennes pour chaque sous-groupe
moyennes_par_categorie = X_r_df.groupby("Sous-groupe d'aliment").mean()

# Créer le graphique à barres groupées
fig, ax = plt.subplots(figsize=(12, 8))

# L'index pour les sous-groupes d'aliments
index = np.arange(len(moyennes_par_categorie))
bar_width = 0.2  # Largeur des barres
space_between_groups = 0.01  # Espace entre les groupes

# Créer une barre pour chaque composante principale
for i, pc in enumerate(["PC1", "PC2", "PC3"]):
    plt.bar(
        index + i * (bar_width + space_between_groups),
        moyennes_par_categorie[pc],
        bar_width,
        label=pc,
    )

# Ajouter des légendes et des titres
ax.set_xlabel("Sous-groupe d'Aliments")
ax.set_ylabel("Moyenne des Composantes Principales")
ax.set_title(
    "Comparaison des Impacts Environnementaux par Sous-groupe d'Aliments (Données ACP)"
)
ax.set_xticks(index + bar_width)
ax.set_xticklabels(moyennes_par_categorie.index)
ax.legend()

plt.xticks(
    rotation=0, ha="right"
)  # Rotation des étiquettes pour une meilleure lisibilité
plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
plt.show()

In [ ]:
# Définir la taille du graphique
plt.figure(figsize=(12, 8))

# Utiliser les scores de la première composante principale comme couleurs
colors = X_r_df["PC1"]

# Tracer les points pour les deux composantes principales
plt.scatter(
    X_r_df.values[:, 0], X_r_df.values[:, 1], s=50, c=colors, cmap="viridis", alpha=0.5
)

# Ajouter des flèches pour les composantes principales et les annoter
for i in range(pca.components_.shape[1]):
    plt.arrow(
        0,
        0,
        pca.components_[0, i] * 12,
        pca.components_[1, i] * 11,
        alpha=0.5,
        head_width=0.1,
        head_length=0.1,
        fc="black",
        ec="black",
    )
    plt.text(
        pca.components_[0, i] * 10 * 1.15,
        pca.components_[1, i] * 10 * 1.15,
        X.columns[i],
        color="green",
        ha="center",
        va="center",
        fontsize=8,
    )

# Ajouter des titres et des étiquettes
plt.title("Biplot des deux premières composantes principales")
plt.xlabel("PC1")
plt.ylabel("PC2")
plt.colorbar()  # Ajouter une barre de couleur pour indiquer l'échelle

# Afficher le graphique
plt.grid(True)
plt.axis("equal")
plt.show()

In [ ]:
# Créer un DataFrame pour les vecteurs propres
loadings = pd.DataFrame(
    pca.components_.T, columns=["PC1", "PC2", "PC3"], index=X.columns
)
loadings

In [ ]:
# Grouper les données par 'Sous-groupe d'aliment' et calculer la moyenne pour les indicateurs sélectionnés
moyennes_par_categorie = df_join_norm.groupby("Sous-groupe d'aliment").mean()


def plot_grouped_bar_chart(data, columns, group):
    # Nombre de groupes d'aliments
    n_groups = len(data.index)

    # Créer le graphique à barres groupées
    fig, ax = plt.subplots(figsize=(8, 5))

    # L'index pour les groupes d'aliments
    index = np.arange(n_groups)
    bar_width = 0.01  # Largeur des barres
    space_between_groups = 0.01  # Espace entre les groupes

    # Créer une barre pour chaque indicateur d'impact sélectionné
    for i, impact in enumerate(columns):
        plt.bar(
            index + i * (bar_width + space_between_groups),
            data[impact],
            bar_width,
            label=impact,
        )

    # Ajouter des légendes et des titres
    ax.set_ylabel("Scores EF Moyens")
    ax.set_title(f"Comparaison des Impacts Environnementaux pour {group}")
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(data.index)
    ax.legend(bbox_to_anchor=(1.24, 2), loc="upper left")

    plt.xticks(
        rotation=0, ha="center"
    )  # Rotation des étiquettes pour une meilleure lisibilité
    plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
    plt.show()


# Sélectionner les colonnes d'intérêt pour la comparaison
colonnes_impact_selectionnees = X.columns
# Utiliser la fonction pour créer le graphique à barres groupées
for group in df_join_norm["Sous-groupe d'aliment"].unique():
    data_subset = moyennes_par_categorie.loc[[group]]
    plot_grouped_bar_chart(data_subset, colonnes_impact_selectionnees, group)

In [ ]:
moyennes_par_categorie

In [ ]:
# Grouper les données par 'Sous-groupe d'aliment' et calculer la moyenne pour les indicateurs sélectionnés
moyennes_par_categorie = df_join_norm.groupby("Sous-groupe d'aliment").mean()


def plot_grouped_bar_chart(data, columns):
    # Nombre de groupes d'aliments
    n_groups = len(data.index)

    # Créer le graphique à barres groupées
    fig, ax = plt.subplots(figsize=(12, 6))

    # L'index pour les groupes d'aliments
    index = np.arange(n_groups)
    bar_width = 0.10  # Largeur des barres
    space_between_groups = 0.01  # Espace entre les groupes

    # Créer une barre pour chaque indicateur d'impact sélectionné
    for i, impact in enumerate(columns):
        plt.bar(
            index + i * (bar_width + space_between_groups),
            data[impact],
            bar_width,
            label=impact,
        )

    # Ajouter des légendes et des titres
    ax.set_xlabel("Sous-groupe d'Aliments")
    ax.set_ylabel("Scores Moyens")
    ax.set_title("Comparaison des Impacts Environnementaux par Sous-groupe d'Aliments")
    ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(moyennes_par_categorie.index)
    ax.legend()

    plt.xticks(
        rotation=0, ha="center"
    )  # Rotation des étiquettes pour une meilleure lisibilité
    plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
    plt.show()


# Sélectionner les colonnes d'intérêt pour la comparaison
colonnes_impact_selectionnees = [
    "Changement climatique",
    "Appauvrissement de la couche d'ozone",
    "Rayonnements ionisants",
    "Formation photochimique d'ozone",
    "Particules fines",
]

# Utiliser la fonction pour créer le graphique à barres groupées
for i in range(0, 3):
    if i == 2:
        colonnes_impact_selectionnees = X.columns[i * 5 :]
    else:
        colonnes_impact_selectionnees = X.columns[i * 5 : (i + 1) * 5]
    plot_grouped_bar_chart(moyennes_par_categorie, colonnes_impact_selectionnees)

In [ ]:
import math


def plot_grouped_bar_chart(ax, data, columns, group, ymin, ymax):
    # Nombre de groupes d'aliments
    n_groups = len(data.index)

    # L'index pour les groupes d'aliments
    index = np.arange(n_groups)
    bar_width = 0.01  # Largeur des barres
    space_between_groups = 0.01  # Espace entre les groupes

    # Créer une barre pour chaque indicateur d'impact sélectionné
    for i, impact in enumerate(columns):
        ax.bar(
            index + i * (bar_width + space_between_groups),
            data[impact],
            bar_width,
            label=impact,
        )

    # Ajouter des légendes et des titres
    ax.set_ylabel("Scores EF Moyens")
    ax.set_title(f"Comparaison des Impacts Environnementaux pour {group}")
    # ax.set_xticks(index + bar_width / 2)
    ax.set_xticklabels(data.index)

    ax.set_xticklabels(
        data.index, rotation=0, ha="center"
    )  # Rotation des étiquettes pour une meilleure lisibilité

    ax.set_ylim([ymin, ymax])  # Set the same y-axis limits for all subplots


# Sélectionner les colonnes d'intérêt pour la comparaison
colonnes_impact_selectionnees = X.columns

# Utiliser la fonction pour créer le graphique à barres groupées
groups = df_join_norm["Sous-groupe d'aliment"].unique()
n_plots = len(groups)
n_cols = 2
n_rows = math.ceil(n_plots / n_cols)

fig, axs = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows), tight_layout=True)

for ax, group in zip(axs.flatten(), groups):
    data_subset = moyennes_par_categorie.loc[[group]]
    plot_grouped_bar_chart(ax, data_subset, colonnes_impact_selectionnees, group, 0, 1)

# Display the legend outside of the plot area and only once for all subplots
handles, labels = axs[0][0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

# Create a grid of subplots
fig = plt.figure(figsize=(15, 15))
gs = gridspec.GridSpec(3, 2, width_ratios=[1, 1])

# Create the first plot on the first row
ax1 = plt.subplot(gs[0, 0])
group = groups[0]
data_subset = moyennes_par_categorie.loc[[group]]
plot_grouped_bar_chart(ax1, data_subset, colonnes_impact_selectionnees, group, 0, 1)

# Create the second and third plots on the second row
for i in range(1, 3):
    ax = plt.subplot(gs[1, i - 1])
    group = groups[i]
    data_subset = moyennes_par_categorie.loc[[group]]
    plot_grouped_bar_chart(ax, data_subset, colonnes_impact_selectionnees, group, 0, 1)

# Create the fourth and fifth plots on the third row
for i in range(3, 5):
    ax = plt.subplot(gs[2, i - 3])
    group = groups[i]
    data_subset = moyennes_par_categorie.loc[[group]]
    plot_grouped_bar_chart(ax, data_subset, colonnes_impact_selectionnees, group, 0, 1)

# Display the legend outside of the plot area and only once for all subplots
handles, labels = ax1.get_legend_handles_labels()
fig.legend(handles, labels, loc="upper right")

plt.tight_layout()
plt.show()

In [ ]:
# Supposons que 'moyennes_par_categorie' est votre DataFrame contenant les moyennes des impacts pour chaque sous-groupe d'aliments
seuil = 0.75

# Créer un dictionnaire pour stocker les impacts principaux pour chaque sous-groupe d'aliments
impacts_principaux = {}

# Parcourir chaque sous-groupe d'aliments
for sous_groupe, impacts in moyennes_par_categorie.iterrows():
    # Appliquer le seuil et récupérer les noms des impacts principaux
    impacts_principaux[sous_groupe] = list(
        impacts[impacts >= seuil * impacts.max()].index
    )

# Convertir le dictionnaire en DataFrame pour un affichage sous forme de tableau
df_impacts_principaux = pd.DataFrame(
    impacts_principaux.items(),
    columns=["Sous-groupe d'aliments", "Impacts Principaux"],
)

# Afficher le DataFrame
df_impacts_principaux

In [ ]:
df_impacts_principaux.iloc[3, 1]

```markdown
### Changement climatique

In [ ]:
df_jointure.groupby("Sous-groupe d'aliment").count()

In [ ]:
df_jointure.columns[39:45]
# 51:57

In [ ]:
df_viande_cuite = df_jointure[df_jointure["Sous-groupe d'aliment"] == "viandes cuites"]
cols = ["Code CIQUAL"]
cols += list(df_jointure.columns[39:45])
df_viande_cuite = df_viande_cuite[cols]
df_viande_cuite

In [ ]:
import scipy.stats as stats

f_value, p_value = stats.f_oneway(
    df_viande_cuite["Changement climatique - Transformation"],
    df_viande_cuite["Changement climatique - Emballage"],
    df_viande_cuite["Changement climatique - Transport"],
    df_viande_cuite["Changement climatique - Supermarché et distribution"],
    df_viande_cuite["Changement climatique - Consommation"],
)

# Afficher les résultats du test ANOVA
print(f"F-value: {f_value}, P-value: {p_value}")

# Si le P-value est inférieur à 0.05, cela indique des différences significatives
if p_value < 0.05:
    print("Des différences significatives existent entre les groupes.")
else:
    print("Aucune différence significative n'a été trouvée entre les groupes.")

In [ ]:
# Calculer la somme ou la moyenne des impacts pour chaque étape
impact_transformation = df_viande_cuite["Changement climatique - Transformation"].mean()
impact_emballage = df_viande_cuite["Changement climatique - Emballage"].mean()
impact_transport = df_viande_cuite["Changement climatique - Transport"].mean()
impact_supermarche = df_viande_cuite[
    "Changement climatique - Supermarché et distribution"
].mean()
impact_consommation = df_viande_cuite["Changement climatique - Consommation"].mean()

# Créer un dictionnaire pour stocker les résultats
impacts = {
    "Transformation": impact_transformation,
    "Emballage": impact_emballage,
    "Transport": impact_transport,
    "Supermarché et distribution": impact_supermarche,
    "Consommation": impact_consommation,
}

# Trouver l'étape avec le plus grand impact
etape_plus_impact = max(impacts, key=impacts.get)

# Afficher l'étape avec le plus grand impact
print(
    f"L'étape ayant le plus grand impact sur le changement climatique est : {etape_plus_impact} avec une moyenne de {impacts[etape_plus_impact]}"
)
impacts

In [ ]:
# Trouver l'étape avec le plus grand impact
etape_plus_impact = max(impacts, key=impacts.get)
max_value = impacts[etape_plus_impact]

# Créer le graphique à barres
plt.figure(figsize=(10, 6))
bars = plt.bar(impacts.keys(), impacts.values(), color="skyblue", width=0.4)

# Ajouter des pointillés pour marquer l'étape avec le plus grand impact
plt.axhline(y=max_value, color="grey", linestyle="--", linewidth=1)
plt.text(
    0.10,
    max_value,
    f"Maximum Impact: {max_value:.2f}",
    color="black",
    va="bottom",
    ha="center",
)

# Mettre en évidence la barre avec le plus grand impact
bars[list(impacts.keys()).index(etape_plus_impact)].set_color("red")

# Ajouter des titres et des étiquettes
plt.title(
    "Impact sur le Changement Climatique par Étape du Cycle de Vie (Viandes Cuites)"
)
plt.xlabel("Étapes du Cycle de Vie")
plt.ylabel("Impact Moyen sur le Changement Climatique")
plt.xticks(
    rotation=0, ha="center"
)  # Rotation des étiquettes pour une meilleure lisibilité

# Afficher le graphique
plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
plt.show()

In [ ]:
def impact_aliment(
    columns=[], sg_aliment="viandes cuites", df_jointure=df_jointure, title=""
):
    df_sg = df_jointure[df_jointure["Sous-groupe d'aliment"] == sg_aliment]
    cols = ["Code CIQUAL"]
    cols += list(columns)
    df_sg = df_sg[cols]
    df_sg

    f_value, p_value = stats.f_oneway(
        df_sg[columns[0]],
        df_sg[columns[1]],
        df_sg[columns[2]],
        df_sg[columns[3]],
        df_sg[columns[4]],
        df_sg[columns[5]],
    )

    # Afficher les résultats du test ANOVA
    print(f"F-value: {f_value}, P-value: {p_value}")

    # Si le P-value est inférieur à 0.05, cela indique des différences significatives
    if p_value < 0.05:
        print("Des différences significatives existent entre les groupes.")
    else:
        print("Aucune différence significative n'a été trouvée entre les groupes.")

    # Calculer la somme ou la moyenne des impacts pour chaque étape
    impact_argiculture = df_sg[columns[0]].mean()
    impact_transformation = df_sg[columns[1]].mean()
    impact_emballage = df_sg[columns[2]].mean()
    impact_transport = df_sg[columns[3]].mean()
    impact_supermarche = df_sg[columns[4]].mean()
    impact_consommation = df_sg[columns[5]].mean()

    # Créer un dictionnaire pour stocker les résultats
    impacts = {
        "Agriculture": impact_argiculture,
        "Transformation": impact_transformation,
        "Emballage": impact_emballage,
        "Transport": impact_transport,
        "Supermarché et distribution": impact_supermarche,
        "Consommation": impact_consommation,
    }

    # Trouver l'étape avec le plus grand impact
    etape_plus_impact = max(impacts, key=impacts.get)

    print(impacts)

    # Afficher l'étape avec le plus grand impact
    print(
        f"L'étape ayant le plus grand impact sur {title} est : {etape_plus_impact} avec une moyenne de {impacts[etape_plus_impact]}"
    )

    # Trouver l'étape avec le plus grand impact
    etape_plus_impact = max(impacts, key=impacts.get)
    max_value = impacts[etape_plus_impact]

    # Créer le graphique à barres
    plt.figure(figsize=(10, 6))
    bars = plt.bar(impacts.keys(), impacts.values(), color="skyblue", width=0.4)

    # Ajouter des pointillés pour marquer l'étape avec le plus grand impact
    plt.axhline(y=max_value, color="grey", linestyle="--", linewidth=1)
    plt.text(
        0.10,
        max_value,
        f"Maximum Impact: {max_value:.2f}",
        color="black",
        va="bottom",
        ha="center",
    )

    # Mettre en évidence la barre avec le plus grand impact
    bars[list(impacts.keys()).index(etape_plus_impact)].set_color("red")

    # Ajouter des titres et des étiquettes
    plt.title(f"Impact sur {title} par Étape du Cycle de Vie ({sg_aliment})")
    plt.xlabel("Étapes du Cycle de Vie")
    plt.ylabel(f"Impact Moyen sur {title}")
    plt.xticks(
        rotation=0, ha="center"
    )  # Rotation des étiquettes pour une meilleure lisibilité

    # Afficher le graphique
    plt.tight_layout()  # Ajuste automatiquement les paramètres de la figure pour donner de l'espace aux étiquettes
    plt.show()


impact_aliment(
    columns=df_jointure.columns[39:45],
    sg_aliment="viandes crues",
    df_jointure=df_jointure,
    title="le Changement Climatique",
)

In [ ]:
df_jointure.columns[51:57]

In [ ]:
impact_aliment(
    columns=df_jointure.columns[51:57],
    sg_aliment="viandes cuites",
    df_jointure=df_jointure,
    title="le Rayonnements ionisants",
)

In [ ]:
df_jointure.columns[81:87]

In [ ]:
impact_aliment(
    columns=df_jointure.columns[81:87],
    sg_aliment="viandes cuites",
    df_jointure=df_jointure,
    title="l'Acidification terrestre et eaux douces",
)

In [ ]:
df_jointure.columns[99:105]

In [ ]:
impact_aliment(
    columns=df_jointure.columns[99:105],
    sg_aliment="viandes crues",
    df_jointure=df_jointure,
    title="l'Eutrophisation terrestre",
)

In [ ]:
df_jointure.columns[111:117]

In [ ]:
impact_aliment(
    columns=df_jointure.columns[111:117],
    sg_aliment="viandes crues",
    df_jointure=df_jointure,
    title="l'Utilisation du sol",
)

In [ ]:
colonnes_selectionnees = [
    "Changement climatique",
    "Effets toxicologiques sur la santé humaine   substances non-cancérogènes",
    "Effets toxicologiques sur la santé humaine   substances cancérogènes",
    "Écotoxicité pour écosystèmes aquatiques d'eau douce",
    "Utilisation du sol",
    "Épuisement des ressources eau",
    "Épuisement des ressources énergétiques",
    "Épuisement des ressources minéraux",
]

# treshold sur la colonne score unique EF
median_value = df_synt["Score unique EF"].median()
labels = np.where(df_synt["Score unique EF"] > median_value, 1, 0)

X = df_synt[colonnes_selectionnees]

print("valeurs 1", len(labels[labels == 1]))
print("valeurs 0", len(labels[labels == 0]))

In [ ]:
# Refaire l'ACP avec les données normalisées
pca = PCA(n_components=2)
X = df_synt[df_synt.columns[13:29]]
print(X.shape)
X_r = pca.fit_transform(X)

# Afficher les dimensions et les proportions de variance expliquée
summary_table = pd.DataFrame(
    columns=[
        "Dimension",
        "Explained Variance",
        "Explained Variance Ratio",
        "Cumulative Explained Variance Ratio",
    ]
)

# Créer une liste pour stocker les lignes du tableau récapitulatif
rows = []

# Calculer les proportions cumulatives de variance expliquée
cumulative_variance_ratio = np.cumsum(pca.explained_variance_ratio_)

# Remplir la liste avec les informations sur chaque dimension
for i in range(len(pca.explained_variance_)):
    rows.append(
        {
            "Dimension": i + 1,
            "Explained Variance": pca.explained_variance_[i],
            "Explained Variance Ratio": pca.explained_variance_ratio_[i],
            "Cumulative Explained Variance Ratio": cumulative_variance_ratio[i],
        }
    )

# Créer le DataFrame récapitulatif en concaténant les lignes
summary_table = pd.concat([pd.DataFrame([row]) for row in rows], ignore_index=True)
summary_table

In [ ]:
print(X_r.shape)

In [ ]:
# Regressions logistiques
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    classification_report,
    confusion_matrix,
    precision_score,
    roc_curve,
    roc_auc_score,
)
from sklearn.model_selection import cross_val_score


def display_results(lr_model, X_test, y_test, y_pred, colonnes_selectionnees):
    # Affichage du rapport de classification
    print(classification_report(y_test, y_pred, target_names=["faibe", "fort"]))

    # Affichage de la matrice de confusion
    conf_mat = confusion_matrix(y_test, y_pred)
    fig, ax = plt.subplots(figsize=(6, 6))
    sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues", cbar=False, ax=ax)
    ax.set_xlabel("Prédit")
    ax.set_ylabel("Vrai")
    ax.set_title("Matrice de confusion")
    ax.xaxis.set_ticklabels(["faible", "fort"])
    ax.yaxis.set_ticklabels(["faible", "fort"])
    plt.show()

    y_pred_proba = lr_model.predict_proba(X_test)[:, 1]

    # Calcul de la courbe ROC
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

    # Calcul de l'aire sous la courbe ROC
    auc = roc_auc_score(y_test, y_pred_proba)

    # Création de la courbe ROC
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, color="darkorange", lw=2, label="ROC curve (area = %0.2f)" % auc)
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("Taux de faux positifs")
    plt.ylabel("Taux de vrais positifs")
    plt.title("Courbe ROC")
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Division des données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    X_r, labels, test_size=0.2, random_state=0
)

# Création du modèle de régression logistique
lr_model = LogisticRegression(max_iter=1000)

# Cross validation
scores = cross_val_score(lr_model, X_train, y_train, cv=5)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

# Entraînement du modèle
lr_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred = lr_model.predict(X_test)

print("acc : ", precision_score(y_test, y_pred))

precision_lr = np.where(y_test == y_pred, 1, 0)

# Affichage des résultats
display_results(lr_model, X_test, y_test, y_pred, colonnes_selectionnees)

In [ ]:
# SVM
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

# Création du modèle SVM
svm_model = SVC(probability=True)

# Paramètres à tester
param_grid = {
    "C": [1],
    "kernel": ["linear"],
}

# Recherche des meilleurs paramètres
grid = GridSearchCV(svm_model, param_grid, refit=True, verbose=2)
grid.fit(X_train, y_train)

# Meilleurs paramètres
print(grid.best_params_)

# Prédiction sur l'ensemble de test
y_pred_svm = grid.predict(X_test)
print("accuracy:", grid.score(X_test, y_test))

precision_svm = np.where(y_test == y_pred_svm, 1, 0)

# Affichage des résultats
display_results(grid, X_test, y_test, y_pred_svm, colonnes_selectionnees)

In [ ]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier

# Création du modèle Random Forest
rf_model = RandomForestClassifier()

# Entraînement du modèle
rf_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_rf = rf_model.predict(X_test)

print("accuracy:", rf_model.score(X_test, y_test))

precision_rf = np.where(y_test == y_pred_rf, 1, 0)

# Affichage des résultats
display_results(rf_model, X_test, y_test, y_pred_rf, colonnes_selectionnees)

In [ ]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier

# Création du modèle Decision Tree
dt_model = DecisionTreeClassifier()

# Entraînement du modèle
dt_model.fit(X_train, y_train)


# Prédiction sur l'ensemble de test
y_pred_dt = dt_model.predict(X_test)
print("accuracy:", dt_model.score(X_test, y_test))
precision_dt = np.where(y_test == y_pred_dt, 1, 0)

# Affichage des résultats
display_results(dt_model, X_test, y_test, y_pred_dt, colonnes_selectionnees)

In [ ]:
# KNN
from sklearn.neighbors import KNeighborsClassifier

# Création du modèle KNN
knn_model = KNeighborsClassifier()

# Entraînement du modèle
knn_model.fit(X_train, y_train)

# Prédiction sur l'ensemble de test
y_pred_knn = knn_model.predict(X_test)
print("accuracy:", knn_model.score(X_test, y_test))
precision_knn = np.where(y_test == y_pred_knn, 1, 0)

# Affichage des résultats
display_results(knn_model, X_test, y_test, y_pred_knn, colonnes_selectionnees)

In [ ]:
def display_roc_curves(models, preds, X_test, y_test):
    import matplotlib.pyplot as plt
    from sklearn.metrics import roc_curve, roc_auc_score

    plt.figure(figsize=(6, 6))

    # Boucle sur les modèles et les prédictions
    for model, y_pred in zip(models, preds):
        # Calcul des probabilités prédites
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calcul de la courbe ROC
        fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

        # Calcul de l'aire sous la courbe ROC
        auc = roc_auc_score(y_test, y_pred_proba)

        # Création de la courbe ROC

        if isinstance(model, GridSearchCV):
            label = "ROC curve (area = %0.2f) for SVM" % auc
        else:
            label = "ROC curve (area = %0.2f) for %s" % (auc, type(model).__name__)

        plt.plot(fpr, tpr, lw=2, label=label)

    # Ajout de la ligne diagonale
    plt.plot([0, 1], [0, 1], color="navy", lw=2, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("Taux de faux positifs")
    plt.ylabel("Taux de vrais positifs")
    plt.title("Courbe ROC")
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
display_roc_curves(
    [lr_model, grid, rf_model, dt_model, knn_model],
    [y_pred, y_pred_svm, y_pred_rf, y_pred_dt, y_pred_knn],
    X_test,
    y_test,
)

In [ ]:
len(precision_svm)

In [ ]:
from scipy.stats import ttest_rel
from sklearn.metrics import precision_score, recall_score
import pandas as pd

model_names = ["Logistic Regression", "SVM", "Random Forest", "Decision Tree", "KNN"]

# Calcul de la précision pour chaque modèle
precisions = [precision_lr, precision_svm, precision_rf, precision_dt, precision_knn]

# Calcul du test t de Student apparié pour chaque paire de modèles
t_stats = []
p_vals = []
model1_list = []
model2_list = []
hypothesis_results = []
alpha = 0.05
for i in range(len(precisions)):
    for j in range(i + 1, len(precisions)):
        t_stat, p_val = ttest_rel(precisions[i], precisions[j])
        model1_list.append(model_names[i])
        model2_list.append(model_names[j])
        t_stats.append(t_stat)
        p_vals.append(p_val)
        hypothesis_results.append("Reject H0" if p_val < alpha else "Fail to reject H0")

# Création d'un DataFrame pour l'affichage
df = pd.DataFrame(
    {
        "Model-1": model1_list,
        "Model-2": model2_list,
        "t-statistic": t_stats,
        "p-value": p_vals,
        "Hypothesis Result": hypothesis_results,
    }
)

display(df)

In [ ]:
models = {
    "Régression Logistique": lr_model,
    "SVM": grid,
    "Random Forest": rf_model,
    "Arbre de Décision": dt_model,
    "KNN": knn_model,
}

In [ ]:
import dash
from dash import dcc, html
import plotly.express as px
import plotly.graph_objs as go
from dash.dependencies import Input, Output
import pandas as pd
import dash_bootstrap_components as dbc
from pyngrok import ngrok

app = dash.Dash(__name__)

app.layout = html.Div(
    children=[
        html.H1(children="Impact Environnemental des Aliments"),
        html.Div(children="""Dashboard interactif basé sur les données Agribalyse."""),
        dcc.Tabs(
            id="tabs",
            children=[
                dcc.Tab(
                    label="Vue d’ensemble",
                    children=[
                        html.Div(
                            [
                                dcc.Graph(
                                    id="histogram",
                                    figure=px.histogram(
                                        df_synt,
                                        x="Score unique EF",
                                        title="Distribution du Score unique EF",
                                    ),
                                )
                            ]
                        )
                    ],
                ),
                dcc.Tab(
                    label="Comparaison par sous-groupe",
                    children=[
                        html.Div(
                            [
                                dcc.Dropdown(
                                    id="dropdown-category",
                                    options=[
                                        {"label": category, "value": category}
                                        for category in df_synt[
                                            "Sous-groupe d'aliment"
                                        ].unique()
                                    ],
                                    value="All",
                                    placeholder="Sélectionnez un sous-groupe d'aliment",
                                ),
                                dcc.Graph(id="bar-chart-category"),
                            ]
                        )
                    ],
                ),
                dcc.Tab(
                    label="Clusters et Scores",
                    children=[
                        html.Div(
                            [
                                dcc.Graph(id="cluster-bar-chart"),
                                dcc.Graph(id="cluster-composition"),
                                dcc.Graph(id="cluster-tsne-plot"),
                            ]
                        )
                    ],
                ),
                dcc.Tab(
                    label="Impact par Étape",
                    children=[
                        html.Div(
                            [
                                dcc.Dropdown(
                                    id="dropdown-subgroup",
                                    options=[
                                        {"label": subgroup, "value": subgroup}
                                        for subgroup in df_eta[
                                            "Sous-groupe d'aliment"
                                        ].unique()
                                    ],
                                    value=None,
                                    placeholder="Sélectionnez un sous-groupe d'aliment",
                                    style={"width": "48%", "display": "inline-block"},
                                ),
                                dcc.Dropdown(
                                    id="dropdown-indicator",
                                    options=[
                                        {
                                            "label": "Changement climatique",
                                            "value": "Changement climatique",
                                        },
                                        {
                                            "label": "Appauvrissement de la couche d'ozone",
                                            "value": "Appauvrissement de la couche d'ozone",
                                        },
                                        {
                                            "label": "Rayonnements ionisants",
                                            "value": "Rayonnements ionisants",
                                        },
                                        {
                                            "label": "Formation photochimique d'ozone",
                                            "value": "Formation photochimique d'ozone",
                                        },
                                        {
                                            "label": "Particules fines",
                                            "value": "Particules fines",
                                        },
                                        {
                                            "label": "Effets toxicologiques sur la santé humaine   substances non-cancérogènes",
                                            "value": "Effets toxicologiques sur la santé humaine   substances non-cancérogènes",
                                        },
                                        {
                                            "label": "Effets toxicologiques sur la santé humaine   substances cancérogènes",
                                            "value": "Effets toxicologiques sur la santé humaine   substances cancérogènes",
                                        },
                                        {
                                            "label": "Écotoxicité pour écosystèmes aquatiques d'eau douce",
                                            "value": "Écotoxicité pour écosystèmes aquatiques d'eau douce",
                                        },
                                        {
                                            "label": "Utilisation du sol",
                                            "value": "Utilisation du sol",
                                        },
                                        {
                                            "label": "Épuisement des ressources eau",
                                            "value": "Épuisement des ressources eau",
                                        },
                                        {
                                            "label": "Épuisement des ressources énergétiques",
                                            "value": "Épuisement des ressources énergétiques",
                                        },
                                        {
                                            "label": "Épuisement des ressources minéraux",
                                            "value": "Épuisement des ressources minéraux",
                                        },
                                    ],
                                    value=None,
                                    placeholder="Sélectionnez un indicateur",
                                    style={
                                        "width": "48%",
                                        "float": "right",
                                        "display": "inline-block",
                                    },
                                ),
                                dcc.Graph(id="step-impact-bar-chart"),
                            ]
                        )
                    ],
                ),
                # Onglet "Modèles"
                dcc.Tab(
                    label="Modèles",
                    children=[
                        html.Div(
                            [
                                html.Label("Choisissez un modèle :"),
                                dcc.RadioItems(
                                    id="model-radio",
                                    options=[
                                        {"label": model, "value": model}
                                        for model in models.keys()
                                    ],
                                    value="Régression Logistique",  # valeur par défaut
                                ),
                            ]
                        ),
                        html.Div(
                            [
                                html.Label("Choisissez les métriques à afficher :"),
                                dcc.Checklist(
                                    id="metric-checklist",
                                    options=[
                                        {
                                            "label": "Matrice de Confusion",
                                            "value": "confusion_matrix",
                                        },
                                        {
                                            "label": "Rapport de Classification",
                                            "value": "classification_report",
                                        },
                                        {"label": "Courbe ROC", "value": "roc_curve"},
                                    ],
                                    value=["roc_curve"],  # valeur par défaut
                                ),
                            ]
                        ),
                        html.Div(id="graph-output"),
                    ],
                ),
            ],
        ),
    ]
)


@app.callback(
    Output("bar-chart-category", "figure"), [Input("dropdown-category", "value")]
)
def update_bar_chart(selected_category):
    filtered_df = (
        df_synt
        if selected_category == "All"
        else df_synt[df_synt["Sous-groupe d'aliment"] == selected_category]
    )
    fig = px.bar(
        filtered_df,
        x="Sous-groupe d'aliment",
        y="Score unique EF",
        title=f"Impact Environnemental par Sous-groupe: {selected_category}",
    )
    return fig


@app.callback(Output("cluster-bar-chart", "figure"), [Input("tabs", "value")])
def update_cluster_bar_chart(_):
    cluster_labels = list(clusters.keys())
    aliments_par_cluster = [len(clusters[cluster]) for cluster in cluster_labels]
    scores_moyens_ef = [moyennes_clusters[cluster] for cluster in cluster_labels]

    fig = go.Figure()

    # Bar chart for the number of foods per cluster
    fig.add_trace(
        go.Bar(
            x=cluster_labels,
            y=aliments_par_cluster,
            name="Nombre d'Aliments",
            marker_color="skyblue",
            yaxis="y1",
        )
    )

    # Line chart for the average EF scores
    fig.add_trace(
        go.Scatter(
            x=cluster_labels,
            y=scores_moyens_ef,
            mode="lines+markers",
            name="Score EF Moyen",
            marker_color="red",
            yaxis="y2",
        )
    )

    fig.update_layout(
        title="Nombre d'Aliments et Score EF Moyen par Cluster",
        xaxis_title="Numéro du Cluster",
        yaxis=dict(
            title="Nombre d'Aliments",
            titlefont=dict(color="skyblue"),
            tickfont=dict(color="skyblue"),
        ),
        yaxis2=dict(
            title="Score EF Moyen",
            titlefont=dict(color="red"),
            tickfont=dict(color="red"),
            overlaying="y",
            side="right",
        ),
        legend=dict(x=0.1, y=0.9),
    )

    return fig


@app.callback(Output("cluster-tsne-plot", "figure"), [Input("tabs", "value")])
def update_tsne_plot(_):
    df_tsne = pd.DataFrame(tsne_data, columns=["x", "y"])
    df_tsne["cluster"] = pred
    fig = px.scatter(
        df_tsne, x="x", y="y", color="cluster", title="TSNE Plot des Clusters"
    )
    return fig


@app.callback(
    Output("step-impact-bar-chart", "figure"),
    [Input("dropdown-subgroup", "value"), Input("dropdown-indicator", "value")],
)
def update_step_impact_chart(selected_subgroup, selected_indicator):
    if not selected_subgroup or not selected_indicator:
        return go.Figure()  # Retourne une figure vide si l'une des valeurs est None
    df_sg = df_eta[df_eta["Sous-groupe d'aliment"] == selected_subgroup]

    col1 = selected_indicator + " - Agriculture"
    col2 = selected_indicator + " - Transformation"
    col3 = selected_indicator + " - Emballage"
    col4 = selected_indicator + " - Transport"
    col5 = selected_indicator + " - Supermarché et distribution"
    col6 = selected_indicator + " - Consommation"

    impacts = {
        "Agriculture": df_sg[col1].mean(),
        "Transformation": df_sg[col2].mean(),
        "Emballage": df_sg[col3].mean(),
        "Transport": df_sg[col4].mean(),
        "Supermarché et distribution": df_sg[col5].mean(),
        "Consommation": df_sg[col6].mean(),
    }

    etape_plus_impact = max(impacts, key=impacts.get)
    max_value = impacts[etape_plus_impact]

    fig = go.Figure()
    fig.add_trace(
        go.Bar(
            x=list(impacts.keys()),
            y=list(impacts.values()),
            marker_color=[
                "skyblue" if step != etape_plus_impact else "red"
                for step in impacts.keys()
            ],
            name="Plus Impactant",
        )
    )

    fig.add_trace(
        go.Scatter(
            x=list(impacts.keys()),
            y=[max_value] * len(impacts),
            mode="lines",
            name="Maximum Impact",
            line=dict(color="grey", dash="dash"),
        )
    )

    fig.update_layout(
        title=f"Impact sur {selected_indicator} par Étape du Cycle de Vie ({selected_subgroup})",
        xaxis_title="Étapes du Cycle de Vie",
        yaxis_title=f"Impact Moyen sur {selected_indicator}",
    )

    return fig


@app.callback(
    Output("cluster-composition", "figure"),
    [
        Input(
            "cluster-bar-chart", "clickData"
        ),  # obtenir les données du cluster cliqué
    ],
)
def update_cluster_bar_chart(clickData):
    if clickData:
        cluster_clicked = clickData["points"][0]["pointIndex"]
        aliments_scores_cluster = clusters[cluster_clicked]

        # Calculer les scores moyens des aliments pour le cluster sélectionné
        cumul_scores_ef = defaultdict(lambda: [0, 0])
        for aliment, score_ef in aliments_scores_cluster:
            cumul_scores_ef[aliment][0] += score_ef
            cumul_scores_ef[aliment][1] += 1

        moyennes_scores_ef = {
            aliment: total_score / count
            for aliment, (total_score, count) in cumul_scores_ef.items()
        }

        aliments_moyennes = sorted(
            moyennes_scores_ef.items(), key=lambda item: item[1], reverse=True
        )
        noms_aliments_moyennes = [aliment for aliment, _ in aliments_moyennes]
        moyennes_ef = [moyenne for _, moyenne in aliments_moyennes]

        fig = go.Figure()
        fig.add_trace(
            go.Bar(
                y=noms_aliments_moyennes,
                x=moyennes_ef,
                orientation="h",
                marker_color="green",
            )
        )

        fig.update_layout(
            title=f"Moyenne de l'Impact Environnemental des Aliments du cluster {cluster_clicked}",
            xaxis_title="Moyenne du Score EF",
            yaxis_title="Aliments",
            yaxis=dict(autorange="reversed"),
        )

        return fig

    # Si aucun cluster n'est sélectionné, renvoyer un graphique vide
    return go.Figure()


@app.callback(
    Output("graph-output", "children"),
    [
        Input("model-radio", "value"),
        Input("metric-checklist", "value"),
    ],
)
def update_graph(selected_model, selected_metrics):
    model = models[selected_model]
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    figures = []

    if "confusion_matrix" in selected_metrics:
        conf_mat = confusion_matrix(y_test, y_pred)
        trace = go.Heatmap(
            z=conf_mat,
            x=["faible", "fort"],
            y=["faible", "fort"],
            colorscale="Blues",
            showscale=False,
        )
        fig = go.Figure(data=trace)
        annotations = []
        for i, row in enumerate(conf_mat):
            for j, value in enumerate(row):
                annotations.append(
                    dict(
                        x=j,
                        y=i,
                        text=str(value),
                        showarrow=False,
                        font=dict(size=14, color="black"),
                    )
                )
        fig.update_layout(
            title="Matrice de Confusion",
            xaxis=dict(title="Prédit", side="bottom"),
            yaxis=dict(title="Vrai", autorange="reversed"),
            annotations=annotations,
        )
        figures.append(dcc.Graph(figure=fig))

    if "classification_report" in selected_metrics:
        # Afficher juste la précision, le recall et le f1-score
        f1_scores = precision_score(y_test, y_pred, average=None)
        precision = precision_score(y_test, y_pred, average=None)
        recall = recall_score(y_test, y_pred, average=None)
        trace = go.Table(
            header=dict(values=["Classe", "Précision", "Rappel", "F1-Score"]),
            cells=dict(values=[["faible", "fort"], precision, recall, f1_scores]),
        )
        fig = go.Figure(trace)
        fig.update_layout(title="Rapport de Classification")
        figures.append(dcc.Graph(figure=fig))

    if "roc_curve" in selected_metrics:
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
        auc = roc_auc_score(y_test, y_pred_proba)
        trace = go.Scatter(
            x=fpr, y=tpr, mode="lines", name=f"Courbe ROC (AUC = {auc:.2f})"
        )
        fig = go.Figure(trace)
        fig.add_trace(
            go.Scatter(
                x=[0, 1],
                y=[0, 1],
                mode="lines",
                line=dict(dash="dash"),
                name="Guess aléatoire",
            )
        )
        fig.update_layout(
            title="Courbe ROC",
            xaxis_title="Taux de faux positifs",
            yaxis_title="Taux de vrais positifs",
        )
        figures.append(dcc.Graph(figure=fig))

    return [dcc.Graph(figure=fig.figure) for fig in figures]


# Ouvrir un tunnel ngrok sur le port 8050
public_url = ngrok.connect(8050)
print("Public URL:", public_url)

# Démarrer le serveur Dash
app.run_server(debug=True, port=8050)